# User behavior analyzer for the user actions captured

First version implemented by: [RumiaTouhou](https://github.com/RumiaTouhou)

Community Contributors: Currently none

This Jupyter Notebook is designed to analyze user interaction data captured by 'user action capturer.py'. This notebook processes and visualizes this data to provide insights into user behavior and interaction patterns. 

### Define the user record to be analyzed

Please define the timestamp of the user record you would like to analyze in the cell below (do not include any suffix). Make sure the json record and the screenshot folder are in the folder same with this notebook.

In [ ]:
# Define User Record Name
user_record_name = ""  # You will need to fill in this

# Example: user_record_name = "2024-05-24-21-19-02"  # This is an example

### Initialization and Setup

The cell below performs the following steps:

1. Locate JSON and Screenshot Folder: The cell tries to find the JSON record file and the corresponding screenshot folder based on the user record name. It checks if these files and folders exist and are not empty. If they are found and are not empty, it prints confirmation messages.

2. Create Analysis Result Folder: The cell creates a new folder named "Analysis Result" within the screenshot folder. If the folder already exists, it removes all existing files in that folder to ensure a clean slate for the new analysis outputs.

3. Import Libraries and Define Variables: The cell imports necessary libraries and defines any variables that might be used later in the implementation of subsequent cells.

In [ ]:


# Step 1: Locate JSON and Screenshot Folder
import os
import json
import shutil
import matplotlib.pyplot as plt

base_directory = os.getcwd()  # Assuming this notebook is in the root folder of the program

# Construct the file paths
json_file_path = os.path.join(base_directory, f"{user_record_name}.json")
screenshot_folder_path = os.path.join(base_directory, f"Screenshots - {user_record_name}")

# Step 2: Check if JSON and Screenshot Folder Exist and Are Not Empty
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"JSON record not found at {json_file_path}")

if os.path.getsize(json_file_path) == 0:
    raise FileNotFoundError(f"JSON record at {json_file_path} is empty")

if not os.path.exists(screenshot_folder_path):
    raise FileNotFoundError(f"Screenshot folder not found at {screenshot_folder_path}")

if not os.listdir(screenshot_folder_path):
    raise FileNotFoundError(f"Screenshot folder at {screenshot_folder_path} is empty")

print(f"JSON record found at {json_file_path} and it is not empty")
print(f"Screenshot folder found at {screenshot_folder_path} and it is not empty")

# Step 3: Create/Empty the "Analysis Result" Folder
analysis_result_folder_path = os.path.join(screenshot_folder_path, "Analysis Result")

# Remove the folder if it exists
if os.path.exists(analysis_result_folder_path):
    shutil.rmtree(analysis_result_folder_path)

# Create the folder
os.makedirs(analysis_result_folder_path)

# Step 4: Import Libraries and Define Variables for Later Use
from collections import defaultdict

# Define constants
THRESHOLD_TIME = 0.15  # seconds
DISTANCE_THRESHOLD = 4  # units in pixels

# Initial variables
window_title_durations = defaultdict(float)
click_counts = defaultdict(int)
click_map_data = defaultdict(list)

# Output initial setup complete message
print("Initial setup complete. Ready for further analysis.")


### Data Integrity Inspection
﻿
The following cell ensures the integrity and correctness of the data captured by the HCI research tool. It performs several checks to validate the JSON record and the corresponding screenshot folder. The specific steps are as follows:

1. Load the JSON Data:

- Loads the JSON data from the specified file path. If there is an error in decoding the JSON file, it raises a ValueError.

2. Check Existence and Non-Emptiness of JSON and Screenshot Folder:

- Verifies that the JSON file and screenshot folder exist and are not empty. If any of these checks fail, it raises a FileNotFoundError.

3. Validate JSON Formatting:

- It ensures that the JSON file contains the required keys (session_start_time, total_duration, initial_foreground_window_title, and actions). If any of these keys are missing, it raises a ValueError.

4. Verify Unique Screenshots for Window Titles:

- Iterates through the recorded window titles in the JSON data and ensures that each window title has a corresponding screenshot file in the screenshot folder. It uses a function to sanitize window titles by replacing invalid characters with valid substitutes. If any screenshot is missing, it raises a FileNotFoundError.

5. Check Screenshot Dimensions:

- It opens each screenshot file and verifies that its dimensions are 1920x1080 pixels. If any screenshot has incorrect dimensions, it raises a ValueError.

6. Validate Drag Durations and Distances:

- For each drag action in the JSON data, the cell calculates the duration and distance of the drag. It ensures that the duration is greater than a specified threshold and that the distance meets the required criteria. If any drag duration or distance is invalid, it raises a ValueError.

7. Check Click Index Sequence:

- The cell verifies that the click indexes in the JSON data are sequential, starting from 1. If the click indexes are not in order, it raises a ValueError.
Validate Total Duration:

It ensures that the total duration recorded in the JSON file is greater than all other timestamps in the actions. If the total duration is not valid, it raises a ValueError. If all these integrity tests pass, the cell prints "Integrity inspections passed!" indicating that the data is valid and ready for further analysis.

In [ ]:
import json
from PIL import Image

# Load the JSON data
try:
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
except json.JSONDecodeError as e:
    raise ValueError(f"Error decoding JSON: {e}")

# Step 1: Check if JSON and Screenshot Folder Exist (Redundant Check)
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"JSON record not found at {json_file_path}")

if os.path.getsize(json_file_path) == 0:
    raise FileNotFoundError(f"JSON record at {json_file_path} is empty")

if not os.path.exists(screenshot_folder_path):
    raise FileNotFoundError(f"Screenshot folder not found at {screenshot_folder_path}")

if not os.listdir(screenshot_folder_path):
    raise FileNotFoundError(f"Screenshot folder at {screenshot_folder_path} is empty")

# Step 2: Validate JSON Formatting
try:
    session_start_time = data["session_start_time"]
    total_duration = data["total_duration"]
    initial_foreground_window_title = data["initial_foreground_window_title"]
    actions = data["actions"]
except KeyError as e:
    raise ValueError(f"Missing key in JSON data: {e}")

# Step 3: Verify Unique Screenshots for Window Titles
INVALID_CHAR_MAPPING = {
    '#': 'SHARP',
    '\\': 'BACKSLASH',
    '/': 'SLASH',
    ':': 'COLON',
    '*': 'ASTERISK',
    '?': 'QUESTION',
    '"': 'QUOTE',
    '<': 'LESS',
    '>': 'GREATER',
    '|': 'PIPE'
}

def sanitize_window_title(window_title):
    """Replace invalid characters in the window title with valid substitutes."""
    for invalid_char, substitute in INVALID_CHAR_MAPPING.items():
        window_title = window_title.replace(invalid_char, substitute)
    return window_title

unique_window_titles = set(action["window_title"] for action in actions)
for title in unique_window_titles:
    sanitized_title = sanitize_window_title(title)
    screenshot_path = os.path.join(screenshot_folder_path, f"{sanitized_title}.png")
    if not os.path.exists(screenshot_path):
        raise FileNotFoundError(f"Screenshot for window title '{title}' not found at {screenshot_path}")

# Step 4: Check Screenshot Dimensions
for title in unique_window_titles:
    sanitized_title = sanitize_window_title(title)
    screenshot_path = os.path.join(screenshot_folder_path, f"{sanitized_title}.png")
    with Image.open(screenshot_path) as img:
        if img.size != (1920, 1080):
            raise ValueError(f"Screenshot {screenshot_path} has invalid dimensions {img.size}, expected 1920x1080")

# Step 5: Validate Drag Durations and Distances
def calculate_distance(start_pos, end_pos):
    """Calculate the Euclidean distance between two points."""
    if isinstance(start_pos, list):
        start_pos = {'x': start_pos[0], 'y': start_pos[1]}
    if isinstance(end_pos, list):
        end_pos = {'x': end_pos[0], 'y': end_pos[1]}
    return ((end_pos['x'] - start_pos['x']) ** 2 + (end_pos['y'] - start_pos['y']) ** 2) ** 0.5

for action in actions:
    if "start_time" in action and "end_time" in action:
        if action["start_time"] is not None and action["end_time"] is not None:
            if action["end_time"] <= action["start_time"]:
                raise ValueError(f"End time is less than or equal to start time in action: {action}")
            
            if "start_coordinates" in action and "end_coordinates" in action:
                start_coordinates = action["start_coordinates"]
                end_coordinates = action["end_coordinates"]
                distance = calculate_distance(start_coordinates, end_coordinates)
                duration = action["end_time"] - action["start_time"]

                if duration < THRESHOLD_TIME:
                    raise ValueError(f"Drag duration {duration} is less than threshold {THRESHOLD_TIME} in action: {action}")

                if distance < DISTANCE_THRESHOLD:
                    raise ValueError(f"Drag distance {distance} is less than threshold {DISTANCE_THRESHOLD} in action: {action}")

# Step 6: Check Click Index Sequence
click_indexes = [action["click_index"] for action in actions]
if sorted(click_indexes) != list(range(1, len(click_indexes) + 1)):
    raise ValueError(f"Click indexes are not sequential: {click_indexes}")

# Step 7: Validate Total Duration
timestamps = [
    action["click_time"] for action in actions if action["click_time"] is not None
] + [
    action["end_time"] for action in actions if action["end_time"] is not None
]

if not timestamps:
    raise ValueError("No valid timestamps found in actions")

max_timestamp = max(timestamps)
if total_duration <= max_timestamp:
    raise ValueError(f"Total duration {total_duration} is not greater than all other timestamps {max_timestamp}")

print("Integrity inspections passed!")


The following cell is use to ensure that the charts generated by matplotlib can display Chinese characters properly. If a font-not-found error was raised in the following cells and there are no non-English characters in the records you previously captured, comment the commands in the cell below.

In [ ]:
# Comment these two lines by adding '#' before the commands if you encounter font problems in the following cells

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### Use Time Analysis

The cell below calculates the time the user spent on each window title and visualizes this data in a pie chart. This analysis helps to understand which applications or windows the user interacted with the most during the session. The steps performed in this cell are as follows:

1. Extract Initial Data:

- The cell extracts the session start time, total duration, initial window title, and the list of actions from the JSON data loaded in Cell 2.

2. Initialize Variables:

- It initializes a dictionary to keep track of the total time spent on each window title.
- It also sets up variables for tracking the current window title and its start time.

3. Calculate Time Spent on Each Window Title:

- The cell iterates through the actions, calculating the duration spent on the current window title until the action's timestamp.
- It updates the dictionary with the calculated duration for each window title and updates the current window title and start time accordingly.
Handle Last Interval:
- After iterating through all actions, the cell calculates the time spent on the final window title up to the total duration of the session and updates the dictionary.

4. Sort and Print Top Window Titles:

- The cell sorts the window titles by the time spent in descending order.
- It prints the top window titles where the user spent the most time. If there are fewer than nine window titles, it prints all of them.

5. Visualize with Pie Chart:

- The cell prepares the data for the pie chart, grouping less significant window titles into an "Others" category if there are more than 10 titles.
- It creates a pie chart using matplotlib to visualize the proportion of time spent on each window title.
- The chart includes a legend, truncating window titles longer than 11 characters.
- The pie chart is displayed in the notebook and saved as an image file in the "Analysis Result" folder.

The pie chart visualizes the distribution of time spent by the user on different window titles during the session. Each slice of the pie represents a window title, with the size of the slice indicating the proportion of time spent on that window.

- **Dominant Applications**: Larger slices indicate the applications or windows where the user spent the most time. This can help identify the primary tools or applications the user interacted with.
- **Application Switching**: If the user spent significant time on multiple applications, it may indicate frequent switching between tasks, which could be analyzed for multitasking behavior or task interruptions.
- **Focus Areas**: Smaller slices or the "Others" category can provide insights into less frequently used applications, which might be secondary tools or less critical to the user's workflow.

In [ ]:

# Step 1: Extract Initial Data
session_start_time = data["session_start_time"]
total_duration = data["total_duration"]
initial_foreground_window_title = data["initial_foreground_window_title"]
actions = data["actions"]

# Step 2: Initialize Variables
window_title_durations = defaultdict(float)
current_window_title = initial_foreground_window_title
current_start_time = 0.0

# Step 3: Calculate Time Spent on Each Window Title
for action in actions:
    if "click_time" in action and action["click_time"] is not None:
        current_end_time = action["click_time"]
    elif "end_time" in action and action["end_time"] is not None:
        current_end_time = action["end_time"]
    else:
        continue
    
    # Calculate duration for current window title
    duration = current_end_time - current_start_time
    window_title_durations[current_window_title] += duration
    
    # Update current window title and start time
    current_window_title = action["window_title"]
    current_start_time = current_end_time

# Step 4: Handle Last Interval
final_duration = total_duration - current_start_time
window_title_durations[current_window_title] += final_duration

# Step 5: Sort and Print Top Window Titles
sorted_window_titles = sorted(window_title_durations.items(), key=lambda item: item[1], reverse=True)
top_window_titles = sorted_window_titles[:9]

print("Top window titles by time spent:")
print("(If there are more than 9 different window titles in the record, then only output the top nine windows sorted by time of use)")
for title, duration in top_window_titles:
    print(f"{title}: {duration:.2f} seconds")

# Step 6: Visualize with Pie Chart
top_titles = [title for title, duration in top_window_titles]
top_durations = [duration for title, duration in top_window_titles]

# If there are more than 10 window titles, group the less significant ones into an “Others” category
if len(sorted_window_titles) > 10:
    others_duration = sum(duration for title, duration in sorted_window_titles[9:])
    top_titles.append("Others")
    top_durations.append(others_duration)

# Create the pie chart
plt.figure(figsize=(14, 10), dpi=300, facecolor='white')  # Increase figure size and resolution
patches, texts, autotexts = plt.pie(
    top_durations, labels=[title if len(title) <= 11 else title[:11] + '...' for title in top_titles],
    autopct=lambda p: f'{p * sum(top_durations) / 100:.3f} sec', startangle=140)

for text in texts:
    text.set_fontsize(8)
for autotext in autotexts:
    autotext.set_fontsize(8)
    autotext.set_color('black')

plt.title("Time Spent on Each Window Title", fontsize=16)

# Add a legend, truncating window titles longer than 11 characters
legend_labels = [
    (title if len(title) <= 11 else title[:11] + '...') + f" - {duration / sum(top_durations) * 100:.1f}%"
    for title, duration in zip(top_titles, top_durations)
]
plt.legend(legend_labels, loc="best", bbox_to_anchor=(1, 0.5), fontsize='small')

# Save the pie chart to the "Analysis Result" folder
pie_chart_path = os.path.join(analysis_result_folder_path, "Time Analysis.png")
plt.savefig(pie_chart_path, bbox_inches="tight", facecolor='white')
plt.show()

print(f"Pie chart saved to {pie_chart_path}")


### Click Count Analysis

The cell below calculates the number of clicks the user performed on each window title and visualizes this data in a bar chart. This analysis helps to understand which applications or windows received the most user interactions in terms of clicks. The steps performed in this cell are as follows:

1. Extract Initial Data:

- The cell extracts the list of actions from the JSON data loaded previously.
Initialize Variables:

- It initializes a dictionary to keep track of the number of clicks for each window title.

2. Calculate Click Counts for Each Window Title:

- The cell iterates through the actions, incrementing the click count for the respective window title for each action.
- If a click or drag causes a window title change, it increments the click count for both the current window title and the new window title.

3. Sort and Print Top Window Titles:

- The cell sorts the window titles by the number of clicks in descending order.
- It prints the top window titles where the user performed the most clicks. If there are fewer than nine window titles, it prints all of them.

4. Visualize with Bar Chart:

- The cell prepares the data for the bar chart, grouping less significant window titles into an "Others" category if there are more than 10 titles.
- It creates a bar chart using matplotlib to visualize the number of clicks for each window title.
Labels are added to the top of each bar indicating the number of clicks, and the font size for window names below each bar is adjusted to be smaller.
- The bar chart is displayed in the notebook and saved as an image file in the "Analysis Result" folder.

The bar chart visualizes the number of clicks the user performed on different window titles during the session. Each bar represents a window title, with the height of the bar indicating the number of clicks.

- **High Interaction Areas**: Taller bars indicate the applications or windows that received the most clicks. This can help identify the areas where the user was most active.
- **Task Intensity**: A high number of clicks on certain applications may suggest task-intensive activities, such as editing, designing, or frequent navigation within the application.
- **Click Distribution**: Comparing the number of clicks across different window titles can reveal how the user's attention and interactions are distributed among various applications.
- **Potential for Optimization**: Applications or windows with an unexpectedly high number of clicks might be candidates for usability improvements, as they may require too many interactions to accomplish tasks.

In [ ]:

# Step 1: Extract Initial Data
actions = data["actions"]

# Step 2: Initialize Variables
click_counts = defaultdict(int)

# Step 3: Calculate Click Counts for Each Window Title
for i in range(len(actions)):
    action = actions[i]
    current_window_title = action["window_title"]
    click_counts[current_window_title] += 1
    
    # Check if the next action changes the window title
    if i + 1 < len(actions):
        next_action = actions[i + 1]
        next_window_title = next_action["window_title"]
        if next_window_title != current_window_title:
            click_counts[next_window_title] += 1

# Step 4: Sort and Print Top Window Titles
sorted_click_counts = sorted(click_counts.items(), key=lambda item: item[1], reverse=True)
top_click_counts = sorted_click_counts[:9]

print("Top window titles by number of clicks:")
for title, count in top_click_counts:
    print(f"{title}: {count} clicks")

# Step 5: Visualize with Bar Chart
top_titles = [title for title, count in top_click_counts]
top_counts = [count for title, count in top_click_counts]

# If there are more than 10 window titles, group the less significant ones into an “Others” category
if len(sorted_click_counts) > 10:
    others_count = sum(count for title, count in sorted_click_counts[9:])
    top_titles.append("Others")
    top_counts.append(others_count)

# Create the bar chart
plt.figure(figsize=(14, 8), dpi=300, facecolor='white')

# Differentiate the "Others" bar color
bar_colors = ['skyblue'] * len(top_counts)
if "Others" in top_titles:
    others_index = top_titles.index("Others")
    bar_colors[others_index] = 'lightcoral'

bars = plt.bar(range(len(top_counts)), top_counts, tick_label=[title if len(title) <= 9 else title[:9] + '...' for title in top_titles], color=bar_colors)

# Add labels to the top of each bar indicating the number of clicks
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.5, int(yval), ha='center', va='bottom', fontsize=10)

plt.xlabel("Window Titles", fontsize=12)
plt.ylabel("Number of Clicks", fontsize=12)
plt.title("Number of Clicks on Each Window Title", fontsize=16)

plt.xticks(fontsize=7)

# Save the bar chart to the "Analysis Result" folder
bar_chart_path = os.path.join(analysis_result_folder_path, "ClickNum Analysis.png")
plt.savefig(bar_chart_path, bbox_inches="tight", facecolor='white')
plt.show()

print(f"Bar chart saved to {bar_chart_path}")


### Click Map Generation

The cell below generates click maps for each screenshot, visualizing the locations and types of clicks and drags performed by the user. These click maps provide a detailed view of user interactions within each application or window. The steps performed in this cell are as follows:

1. Extract Initial Data:

- The cell extracts the list of actions from the JSON data loaded in Cell 2.

2. Initialize Variables and Functions:

- It initializes a dictionary to store click and drag data for each window title.
- A function is defined to sanitize window titles by replacing invalid characters with valid substitutes.

3. Collect Click and Drag Data:

- The cell iterates through the actions, collecting the coordinates, types, and timestamps of clicks and drags.
- It ensures that if a click or drag causes a window title change, the click or drag is counted for both the window title before and after the change.

4. Load Screenshots and Create Click Maps:

- For each unique window title, the cell sanitizes the title to get the corresponding screenshot filename.
- It loads the screenshot and plots the click and drag data directly on the screenshot, using different colors and markers for different types of clicks and drags:
    - Left Clicks: Orange circle dots with yellow grid.
    - Right Clicks: Medium purple circle dots with dark red grid.
    - Left Drags: Light blue dots with dark blue grids for start points, dark blue dots with light blue grids for end points, connected by a blue line.
    - Right Drags: Light green dots with dark green grids for start points, dark green dots with light green grids for end points, connected by a green line.
- The click maps are saved as image files in the "Analysis Result" folder. **Please note that the click maps generated will not be printed here!** 

Click maps provide a visual representation of user interactions within each application or window, showing where and how the user engaged with the interface.

- **Interaction Hotspots**: Clusters of clicks can reveal areas of high interaction, indicating frequently used features or buttons. This helps identify critical parts of the interface.
- **Drag Patterns**: The start and end points of drags, connected by lines, can show how users perform drag-and-drop operations, resize windows, or scroll through content.
- **Usability Issues**: Areas with a high density of clicks or drags might indicate usability issues, such as difficult-to-use controls or inefficient navigation paths.
- **User Behavior**: By analyzing the spatial distribution of clicks and drags, researchers can understand how users navigate through the application, their typical workflows, and any repetitive actions.

Below is an example of click map generated:

![Example Output](ExampleOutput.png)


In [ ]:
from PIL import Image, ImageDraw

# Step 1: Extract Initial Data
actions = data["actions"]

# Step 2: Initialize Variables and Functions
click_map_data = defaultdict(list)

INVALID_CHAR_MAPPING = {
    '#': 'SHARP',
    '\\': 'BACKSLASH',
    '/': 'SLASH',
    ':': 'COLON',
    '*': 'ASTERISK',
    '?': 'QUESTION',
    '"': 'QUOTE',
    '<': 'LESS',
    '>': 'GREATER',
    '|': 'PIPE'
}

def sanitize_window_title(window_title):
    """Replace invalid characters in the window title with valid substitutes."""
    for invalid_char, substitute in INVALID_CHAR_MAPPING.items():
        window_title = window_title.replace(invalid_char, substitute)
    return window_title

# Step 3: Collect Click and Drag Data with Debug Outputs
for i in range(len(actions)):
    action = actions[i]
    click_type = action["click_type"]
    current_window_title = action["window_title"]
    sanitized_title = sanitize_window_title(current_window_title)
    
    print(f"Processing action: {action}")  # DEBUG output
    
    if "click_time" in action and action["click_time"] is not None:
        coords = action["click_coordinates"]
        click_map_data[sanitized_title].append((coords["x"], coords["y"], click_type, "click"))
    elif "start_coordinates" in action and "end_coordinates" in action:
        start_coords = action["start_coordinates"]
        end_coords = action["end_coordinates"]
        print(f"Start coords: {start_coords}, End coords: {end_coords}")  # DEBUG output
        
        if isinstance(start_coords, list):
            start_x, start_y = start_coords[0], start_coords[1]
        else:
            start_x, start_y = start_coords["x"], start_coords["y"]
        
        if isinstance(end_coords, list):
            end_x, end_y = end_coords[0], end_coords[1]
        else:
            end_x, end_y = end_coords["x"], end_coords["y"]
        
        click_map_data[sanitized_title].append((start_x, start_y, click_type, "start"))
        click_map_data[sanitized_title].append((end_x, end_y, click_type, "end"))

# Step 4: Load Screenshots and Create Click Maps
for title in click_map_data.keys():
    sanitized_title = sanitize_window_title(title)
    screenshot_path = os.path.join(screenshot_folder_path, f"{sanitized_title}.png")
    image = Image.open(screenshot_path)
    draw = ImageDraw.Draw(image)
    
    for x, y, click_type, point_type in click_map_data[title]:
        if click_type == "left_click":
            draw.ellipse((x-8, y-8, x+8, y+8), fill='orange', outline='yellow', width=2)
        elif click_type == "right_click":
            draw.ellipse((x-8, y-8, x+8, y+8), fill='mediumpurple', outline='darkred', width=2)
        elif click_type == "left_drag":
            if point_type == "start":
                draw.ellipse((x-8, y-8, x+8, y+8), fill='lightblue', outline='darkblue', width=2)
            elif point_type == "end":
                draw.ellipse((x-8, y-8, x+8, y+8), fill='darkblue', outline='lightblue', width=2)
                draw.line((prev_x, prev_y, x, y), fill='paleturquoise', width=3)
            prev_x, prev_y = x, y
        elif click_type == "right_drag":
            if point_type == "start":
                draw.ellipse((x-8, y-8, x+8, y+8), fill='lightgreen', outline='darkgreen', width=2)
            elif point_type == "end":
                draw.ellipse((x-8, y-8, x+8, y+8), fill='darkgreen', outline='lightgreen', width=2)
                draw.line((prev_x, prev_y, x, y), fill='lawngreen', width=3)
            prev_x, prev_y = x, y
    
    click_map_path = os.path.join(analysis_result_folder_path, f"M - {sanitized_title}.png")
    image.save(click_map_path)

print("Click maps created and saved.")


In [ ]:
print("All done!")

If you like my program, please star it on Github. If you find any potential bugs or glad to refine its functionalities, pull requests are welcomed. 